## Import necessary packages:

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import average_precision_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tqdm import tqdm

seed = 0

Probe = ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']

DoS = ['apache2', 'back', 'land', 'mailbomb', 'neptune', 'pod', 'processtable', 'smurf', 'teardrop', 'udpstorm']

R2L = ['buffer_overflow', 'httptunnel', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm']

U2R = ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'named', 'phf', 'sendmail', 'snmpgetattack', 'snmpguess',
       'spy', 'warezclient', 'warezmaster', 'worm', 'xlock', 'xsnoop']

## Import data and assign feature name to each column:

In [ ]:
columns = []
with open('kddcup.names') as f:
    next(f)
    for line in f:
        feature = line.split(':')[0]
        columns.append(feature)
columns.append('label')

In [ ]:
train_data = pd.read_csv('kddcup.data', header=None)
train_data.columns = columns
train_data['label'] = train_data['label'].apply(lambda x: x.replace('.', ''))

test_data = pd.read_csv('corrected', header=None)
test_data.columns = columns
test_data['label'] = test_data['label'].apply(lambda x: x.replace('.', ''))

In [ ]:
boundary = train_data.shape[0]
whole_data = pd.concat([train_data, test_data], axis=0)

## One-hot encoding:

In [ ]:
categorical = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']
whole_data_categorical = whole_data[categorical].copy()
whole_data.drop(columns=categorical, inplace=True)
whole_data_categorical_T = pd.get_dummies(whole_data_categorical)
whole_data = pd.concat([whole_data_categorical_T, whole_data], axis=1)

## Split data into training and test set:

In [ ]:
train_data = whole_data.iloc[:boundary]
test_data = whole_data.iloc[boundary:]

X_train, y_train = train_data.drop(columns='label'), train_data['label']
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed)
X_test, y_test = test_data.drop(columns='label'), test_data['label']

## Convert specific attack types to four general attack types in y_valid and y_test:

In [ ]:
for attack in Probe:
    y_valid = y_valid.replace(attack, 'Probe')
    y_test = y_test.replace(attack, 'Probe')
    
for attack in DoS:
    y_valid = y_valid.replace(attack, 'DoS')
    y_test = y_test.replace(attack, 'DoS')
    
for attack in R2L:
    y_valid = y_valid.replace(attack, 'R2L')
    y_test = y_test.replace(attack, 'R2L')
    
for attack in U2R:
    y_valid = y_valid.replace(attack, 'U2R')
    y_test = y_test.replace(attack, 'U2R')

## Store preprocessed data for later usage:

In [ ]:
X_train.to_pickle("X_train.pkl")
y_train.to_pickle("y_train.pkl")

X_valid.to_pickle("X_valid.pkl")
y_valid.to_pickle("y_valid.pkl")

X_test.to_pickle("X_test.pkl")
y_test.to_pickle("y_test.pkl")

## Load preprocessed data back:

In [2]:
X_train = pd.read_pickle("X_train.pkl")
y_train = pd.read_pickle("y_train.pkl")

X_valid = pd.read_pickle("X_valid.pkl")
y_valid = pd.read_pickle("y_valid.pkl")

X_test = pd.read_pickle("X_test.pkl")
y_test = pd.read_pickle("y_test.pkl")

## (new version) Sample a subset of training data from the original training set:

In [3]:
# Recombine X_train and y_train into a dataframe:
X_train_df = pd.concat([X_train, y_train], axis=1)

In [4]:
# Divide instances by label:
label_list = np.unique(X_train_df['label'])

df_label_list_dict = {'Probe': [], 'DoS': [], 'R2L': [], 'U2R': [], 'normal': []}
for label in label_list:
    df_label = X_train_df[X_train_df['label'] == label]
    
    if label in Probe:
        df_label_list_dict['Probe'].append(df_label)
    elif label in DoS:
        df_label_list_dict['DoS'].append(df_label)
    elif label in R2L:
        df_label_list_dict['R2L'].append(df_label)
    elif label in U2R:
        df_label_list_dict['U2R'].append(df_label)
    else:
        df_label_list_dict['normal'].append(df_label)

In [5]:
# Determine the maximum number of instances to be sampled from each label:
num_samples_per_label = 500

X_train_df_sampled = pd.DataFrame()
sample_weights = []

for cls, df_label_list in df_label_list_dict.items():
    sample_distribution_within_cls = []
    
    for df_label in df_label_list:
        num_instances = df_label.shape[0]
        num_samples = min(num_instances, num_samples_per_label)
        
        sample_distribution_within_cls.append(num_samples)
        df_sampled = df_label.sample(n=num_samples, random_state=seed)
        X_train_df_sampled = pd.concat([X_train_df_sampled, df_sampled], axis=0)
        
    sample_distribution_within_cls = np.array(sample_distribution_within_cls)
    weight_per_sample_group = sample_distribution_within_cls.sum() / sample_distribution_within_cls
    
    for i in range(len(sample_distribution_within_cls)):
        sample_weights += [weight_per_sample_group[i]] * sample_distribution_within_cls[i]

In [6]:
# Decompose X_train_df_sampled into X_train and y_train again:
X_train, y_train = X_train_df_sampled.drop(columns='label'), X_train_df_sampled['label']

## Sample a subset of training data from the original training set:

In [ ]:
# Recombine X_train and y_train into a dataframe:
X_train_df = pd.concat([X_train, y_train], axis=1)

In [ ]:
# Divide instances by label:
label_list = np.unique(X_train_df['label'])

df_label_list = []
for label in label_list:
    df_label_list.append(X_train_df[X_train_df['label'] == label])

In [ ]:
# Determine how many instances are sampled from each label:
num_samples_per_label = 100
X_train_df_sampled = pd.DataFrame()

for df in df_label_list:
    num_instances = df.shape[0]
    
    replace = False
    if num_instances < num_samples_per_label:
        replace = True
    
    df_sampled = df.sample(n=num_samples_per_label, replace=replace, random_state=seed)
    X_train_df_sampled = pd.concat([X_train_df_sampled, df_sampled], axis=0)

In [ ]:
# Decompose X_train_df_sampled into X_train and y_train again:
X_train, y_train = X_train_df_sampled.drop(columns='label'), X_train_df_sampled['label']

## Convert specific attack types to four general attack types in y_train:

In [7]:
for attack in Probe:
    y_train = y_train.replace(attack, 'Probe')
    
for attack in DoS:
    y_train = y_train.replace(attack, 'DoS')
    
for attack in R2L:
    y_train = y_train.replace(attack, 'R2L')
    
for attack in U2R:
    y_train = y_train.replace(attack, 'U2R')

## Standardize feature vectors using StandardScaler:

In [8]:
sscaler = preprocessing.StandardScaler().fit(X_train)
X_train, X_valid, X_test = sscaler.transform(X_train), sscaler.transform(X_valid), sscaler.transform(X_test)

D:\Anaconda\envs\dme\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\envs\dme\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  
D:\Anaconda\envs\dme\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  
D:\Anaconda\envs\dme\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  


## Use linear PCA to perform dimensionality reduction:
**Warning**: linear PCA **works** with StandardScaler but **doesn't work** with RobustScaler.

In [ ]:
pca = PCA(n_components=X_train.shape[1], svd_solver='full')
pca.fit(X_train)

fig, ax1 = plt.subplots(figsize=(8, 4))
graph1 = ax1.bar(np.arange(X_train.shape[1]), pca.explained_variance_ratio_, label='individual explained variance')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_xlabel('Number of Principal Components')
ax1.set_ylabel('Explained Variance Ratio')

ax2 = ax1.twinx()
graph2 = ax2.plot(np.cumsum(pca.explained_variance_ratio_), c='red', label='cumulative explained variance')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylabel('Explained Variance Ratio')

index_linear = np.nonzero(np.cumsum(pca.explained_variance_ratio_) >= 0.9)[0][0]
graph3 = ax2.vlines(index, 0, 1, color='purple', linestyles="dotted", label='90% variance is explained with {} components'.format(index+1))

graphs = [graph1, graph2[0], graph3]
labels = [g.get_label() for g in graphs]
ax1.legend(graphs, labels, bbox_to_anchor=(1, 0.5))

plt.title('Explained Variance Ratio versus Number of Principal Components')
plt.show()

## Use Kernel PCA to perform dimensionality reduction

In [ ]:
pca = KernelPCA(n_components=X_train.shape[1], kernel='rbf')
pca.fit_transform(X_train)

fig, ax1 = plt.subplots(figsize=(8, 4))
graph1 = ax1.bar(np.arange(X_train.shape[1]), pca.lambdas_, label='individual eigen values')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_xlabel('Number of Principal Components')
ax1.set_ylabel('Explained Variance Ratio')

ax2 = ax1.twinx()
graph2 = ax2.plot(np.cumsum(pca.lambdas_), c='red', label='cumulative explained variance')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylabel('Explained Variance Ratio')

index_kernel = np.nonzero(np.cumsum(pca.lambdas_) >= sum(pca.lambdas_)*0.9)[0][0]
graph3 = ax2.vlines(index, 0, 1, color='purple', linestyles="dotted", label='90% variance is explained with {} components'.format(index+1))

graphs = [graph1, graph2[0], graph3]
labels = [g.get_label() for g in graphs]
ax1.legend(graphs, labels, bbox_to_anchor=(1, 0.5))

plt.title('Explained Variance Ratio versus Number of Principal Components')
plt.show()

## Batch PCA. Aim to reduce memory usage

In [9]:
def batchPCA(pca, data, batchSize, index):
    instance_num = data.shape[0]
    batch_num = instance_num // batchSize
    pca_result = np.zeros((1,index+1))
    
    for _ in tqdm(range(batch_num)):
        batch = data[0:batchSize]
        batch_pca = pca.transform(batch)
        pca_result = np.concatenate((pca_result, batch_pca), axis=0)
        data = np.delete(data, np.s_[0:batchSize], axis=0)
    
    pca_result = pca_result[1:]
    batch_pca = pca.transform(data)
    pca_result = np.concatenate((pca_result, batch_pca), axis=0)  
    return pca_result

In [11]:
# Transform data using PCA with the optimal number of components found above:
pca = KernelPCA(n_components=index_kernel+1, kernel='rbf')
pca.fit(X_train)
X_train = pca.transform(X_train)
X_valid = batchPCA(pca, X_valid, 20000, index)
X_test = batchPCA(pca, X_test, 20000, index)

100%|████████████████████████████████████████████████████████████████████| 15/15 [00:31<00:00,  2.06s/it]


## Stratified holdout validation:

In [12]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

## Stratified Cross Validation:

In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

## Softmax Regression for multinomial classification:
### Linear decision boundary:

In [15]:
parameters = {
    'C': np.logspace(-5, 5, 30)
}

gs_lgr = GridSearchCV(LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='lbfgs'), parameters, cv=skf)
gs_lgr.fit(X_train, y_train)

lgr = gs_lgr.best_estimator_
y_pred_valid = lgr.predict_proba(X_valid)

# Multiclass average PR_AUC:
avg_PR = 0
for i, label in enumerate(lgr.classes_):
    y_valid_label = (y_valid == label).astype(int)
    y_pred_valid_label = y_pred_valid[:, i]
    avg_PR += average_precision_score(y_valid_label, y_pred_valid_label)
    
avg_PR /= len(lgr.classes_)

print ("The best average PR_AUC for Logistic Regression on validation set is {:.4f}".format(avg_PR))
display(gs_lgr.best_params_)

D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anacond

D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\envs\dme\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anacond

The best average PR_AUC for Logistic Regression on validation set is 0.5972


{'C': 1887.3918221350996}

## Plot confusion matrix of Softmax Regression on validation set:

In [16]:
y_pred_valid = lgr.predict(X_valid)
cm = confusion_matrix(y_valid, y_pred_valid)
cm = pd.DataFrame(data=cm, index=[lgr.classes_], columns=[lgr.classes_])

In [17]:
percent_correct_horizontal = cm.sum(axis=1)
for i in range(cm.shape[0]):
    percent_correct_horizontal.iloc[i] = cm.iloc[i, i] / percent_correct_horizontal.iloc[i]
percent_correct_horizontal = round(percent_correct_horizontal * 100, 2)
percent_correct_horizontal = percent_correct_horizontal.apply(lambda x: str(x) + '%')

In [18]:
percent_correct_vertical = cm.sum(axis=0)
for i in range(cm.shape[0]):
    percent_correct_vertical.iloc[i] = cm.iloc[i, i] / percent_correct_vertical.iloc[i]
percent_correct_vertical = round(percent_correct_vertical * 100, 2)
percent_correct_vertical = percent_correct_vertical.apply(lambda x: str(x) + '%')

In [ ]:
cm['Recall%'] = percent_correct_horizontal
percent_correct_vertical.name = ('Precision%', 5)
cm = cm.append(percent_correct_vertical)
display(cm)

## SVM with radial basis function (rbf) kernel for OvR multiclass classification:
### Non-linear decision boundary:

In [21]:
parameters = {
    'C': np.logspace(-5, 5, 10),
    'gamma': np.logspace(-5, 0, 5)
}

gs_svm_rbf = GridSearchCV(SVC(kernel='rbf', decision_function_shape='ovr', class_weight='balanced'), parameters, cv=sss)
gs_svm_rbf.fit(X_train, y_train)

svm_rbf = gs_svm_rbf.best_estimator_
y_pred_valid = svm_rbf.decision_function(X_valid)

# Multiclass average PR_AUC:
avg_PR = 0
for i, label in enumerate(svm_rbf.classes_):
    y_valid_label = (y_valid == label).astype(int)
    y_pred_valid_label = y_pred_valid[:, i]
    avg_PR += average_precision_score(y_valid_label, y_pred_valid_label)
    
avg_PR /= len(svm_rbf.classes_)

print ("The best average PR_AUC for Support Vector Machine with rbf kernel on validation set is {:.4f}".format(avg_PR))
display(gs_svm_rbf.best_params_)

The best average PR_AUC for Support Vector Machine with rbf kernel on validation set is 0.5902


{'C': 7742.636826811277, 'gamma': 1.0}

## Plot confusion matrix of SVM on validation set:

In [22]:
y_pred_valid = svm_rbf.predict(X_valid)
cm = confusion_matrix(y_valid, y_pred_valid)
cm = pd.DataFrame(data=cm, index=[svm_rbf.classes_], columns=[svm_rbf.classes_])

In [23]:
percent_correct_horizontal = cm.sum(axis=1)
for i in range(cm.shape[0]):
    percent_correct_horizontal.iloc[i] = cm.iloc[i, i] / percent_correct_horizontal.iloc[i]
percent_correct_horizontal = round(percent_correct_horizontal * 100, 2)
percent_correct_horizontal = percent_correct_horizontal.apply(lambda x: str(x) + '%')

In [24]:
percent_correct_vertical = cm.sum(axis=0)
for i in range(cm.shape[0]):
    percent_correct_vertical.iloc[i] = cm.iloc[i, i] / percent_correct_vertical.iloc[i]
percent_correct_vertical = round(percent_correct_vertical * 100, 2)
percent_correct_vertical = percent_correct_vertical.apply(lambda x: str(x) + '%')

In [ ]:
cm['Recall%'] = percent_correct_horizontal
percent_correct_vertical.name = ('Precision%', 5)
cm = cm.append(percent_correct_vertical)
display(cm)

## Ultimate Pipeline :-)

In [ ]:
# Search all combinations of linear and kernel pca with softmax and svm 

pipeline = Pipeline([
    # the reduce_dim stage is populated by the param_grid
    ('reduce_dim', None),
    ('classify', None)
])

# To be determined
gamma_search = np.logspace(-5, 0, num=10)
#componets_search = []

param_grid = [
    # Linear PCA with softmax
    {
        'reduce_dim': [PCA(kernel='linear', n_components=index_linear+1, svd_solver='full')],
        #'reduce_dim__n_components': componets_search,
        'classify': [LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='lbfgs')],
        'classify__C': np.logspace(-5, 5, 30)
    },
    # Linear PCA with kernel SVM
    {
        'reduce_dim': [PCA(kernel='linear', n_components=index_linear+1, svd_solver='full')],
        #'reduce_dim__n_component': componets_search,
        'classify': [SVC(kernel='rbf', decision_function_shape='ovr', class_weight='balanced')],
        'classify__C': np.logspace(-5, 5, 10),
        'classify__gamma': np.logspace(-5, 0, 5)
    },
    # Kernel PCA with softmax
    {
        'reduce_dim': [KernelPCA(kernel='rbf', n_componets=index_kernel+1)],
        #'reduce_dim__n_components': componets_search,
        'reduce_dim__gamma': gamma_search,
        'classify': [LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='lbfgs')],
        'classify__C': np.logspace(-5, 5, 30)
    },
    # Kernel PCA with kernel SVM
    {
        'reduce_dim': [KernelPCA(kernel='rbf', n_components=index_kernel+1)],
        #'reduce_dim__n_component': componets_search,
        'reduce_dim__gamma': gamma_search,
        'classify': [SVC(kernel='rbf', decision_function_shape='ovr', class_weight='balanced')],
        'classify__C': np.logspace(-5, 5, 10),
        'classify__gamma': np.logspace(-5, 0, 5)
    },
]

# Get best combination of pca and classifier
grid = GridSearchCV(pipeline, cv=skf, n_jobs=-1, param_grid=param_grid)
grid.fit(X_train, y_train) # Note this part should use untransformed data
display(grid.best_params_)

# Best PCA
best_combination = grid.best_estimatior
best_pca = best_combination.named_steps['reduce_dim']
best_pca.fit(X_train)

X_train = best_pca.transform(X_train)
X_valid = batchPCA(best_pca, X_valid, 20000, index)
X_test = batchPCA(best_pca, X_test, 20000, index)

# Best classifer
best_clf = best_combination.named_steps['classify']
y_pred_valid = best_clf.predict_proba(X_valid)


# Multiclass average PR_AUC:
avg_PR = 0
for i, label in enumerate(best_clf.classes_):
    y_valid_label = (y_valid == label).astype(int)
    y_pred_valid_label = y_pred_valid[:, i]
    avg_PR += average_precision_score(y_valid_label, y_pred_valid_label)
    
overall_best /= len(overall_best.classes_)

print ("The best average PR_AUC for Logistic Regression on validation set is {:.4f}".format(avg_PR))
display(overall_best.best_params_)